In [1]:
from bs4 import BeautifulSoup as bs4
import requests
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager

from fake_useragent import UserAgent

pages=[
    "https://observador.pt/seccao/observador/fact-check/"
]

urls= set()
podcasts = set()

In [33]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:01<00:00, 3.76MB/s]


In [35]:
for url in pages:
    # Open webpage
    driver.get(url)

    # Scroll down until the end of the webpage to load all news articles
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True

    # Extract the remaining urls
    soup = bs4(driver.page_source, 'html.parser')
    link_elements = soup.find_all("h1")

    for element in link_elements:
        soup = bs4(str(element), 'html.parser')
        link = soup.find('a', class_='obs-accent-color')
        #print(link)
        try:
            audio_element = soup.find('svg', class_ = 'obs-icon obs-icon-volume obs-accent-fill-color')
            url = link.get('href')
            if audio_element:
                print("Podcast:",url)
                podcasts.add(url)
            else:
                print(url)
                urls.add(url)
        except:
            pass

Podcast: https://observador.pt/programas/fact-check-2/tripulantes-do-titan-gravaram-video-de-despedida/
Podcast: https://observador.pt/programas/fact-check-2/usar-telemovel-as-escuras-causa-cancro-nos-olhos/
https://observador.pt/factchecks/fact-check-casca-da-maca-tem-celulas-cancerigenas/
https://observador.pt/factchecks/fact-check-loja-de-retalho-target-pos-a-venda-peca-de-roupa-infantil-com-referencias-satanicas/
https://observador.pt/factchecks/fact-check-imagem-mostra-arvore-em-vias-de-extincao-da-qual-so-existem-tres-exemplares-na-china/
https://observador.pt/factchecks/fact-check-video-de-despedida-da-tripulacao-do-submersivel-titan-foi-revelado/
Podcast: https://observador.pt/programas/fact-check-2/salazar-criou-o-servico-nacional-de-saude/
https://observador.pt/factchecks/fact-check-migracao-ilegal-premiada-na-uniao-europeia-com-trabalho-renda-e-cartao-de-credito/
https://observador.pt/factchecks/fact-check-lei-da-california-permite-infanticidio-ate-aos-28-dias-de-vida-de-um-

In [ ]:
driver.quit()

In [53]:
len(urls)

2577

In [54]:
len(podcasts)

571

In [57]:
urls_test = [
    "https://observador.pt/factchecks/fact-check-passos-coelho-recebe-subvencao-vitalicia/",
]

In [58]:
texts=[]
sources=[]
labels = []
df_urls = []
# Initialize an empty dataframe
df = pd.DataFrame(columns=['Text', 'Source', 'Label', 'URL'])

for url in urls:
    print("-------------")
    print(url)
    print("-------------")
    text = "NONE"
    source = "NONE"
    label = "NONE"
    a_tese_check = False
    article_blocked = False
    try:
        user_agent = UserAgent()
        headers = {'User-Agent': user_agent.random}
        time.sleep(1)
        page = requests.get(url, headers = headers)
        time.sleep(1)
        soup = bs4(page.text, 'html.parser')

        a_frase = soup.find("p", class_ = "factcheck-text")
        h2_elements = soup.find_all("h2")
        for element in h2_elements:
            if element.get_text() == "A tese":
                a_tese_check = True
        p_elements = soup.find_all("p")
        for element in p_elements:
            #print(element)
            if element.get_text() == "Para continuar a ler este artigo, torne-se assinante":
                article_blocked = True
        if a_frase:
            print("a_frase")
            text = a_frase.get_text().strip()
            source = soup.find("p", class_ = "factcheck-reference").get_text().strip()
            label = soup.find("div", class_ = "factcheck-result-text").get_text()
        elif a_tese_check:
            print("a_tese")
            text = soup.find("blockquote").get_text().strip()
            next_element = soup.find("blockquote").find_next_sibling() # The next sibling element has the source
            source = next_element.get_text()
            div_element = soup.select('div[class*="fact-check_"]')
            if div_element:
                label = div_element[0].find("strong").get_text()

        if not(article_blocked):
            print(text)
            print(source)
            print(label)
            
            texts.append(text)
            sources.append(source)
            labels.append(label)
            df_urls.append(url)
        else:
            print("article blocked")
    except:
        pass

    time.sleep(3)

# Create a new dataframe from the lists
new_data = pd.DataFrame({'Text': texts, 'Source': sources, 'Label': labels, 'URL': df_urls})

# Concatenate the new dataframe with the existing dataframe
df = pd.concat([df, new_data], ignore_index=True)

-------------
https://observador.pt/factchecks/fact-check-boletins-de-voto-em-le-pen-foram-escondidos-para-evitar-a-vitoria/
-------------
a_frase
"Policia Francesa encontra equipe responsavel de esconder as cedulas que elegeriam Mariane Le Pen. CONFIRMADA FRAUDE para eleger MACRON."
— Utilizador do Facebook,     13 maio 2022
Errado
-------------
https://observador.pt/factchecks/fact-check-fotografia-mostra-mulheres-portuguesas-que-mataram-familiares-e-que-estao-livres/
-------------
a_frase
"Matou a enteada de cinco anos e está livre. Matou pai e mãe e está livre. Matou o assassino do filho e está preso. Exemplo da lei que temos"
— Utilizador do Facebook,     11 maio 2020
Errado
-------------
https://observador.pt/factchecks/fact-check-fotografia-mostra-jovem-que-ficou-em-segundo-lugar-nas-olimpiadas-de-fisica-quantica/
-------------
a_frase
Este rapaz chama-se Tiago Santos e é natural de Torre de Moncorvo. Tem 19 anos e representou Portugal nas olimpíadas de física quântica tendo fic

In [59]:
# list was 4044
len(urls)

2577

In [60]:
len(df_urls)

2576

In [61]:
len(texts)

2576

In [62]:
len(labels)

2576

In [63]:
len(sources)

2576

In [64]:
df.head()

,Text,Source,Label,URL
0,"""Policia Francesa encontra equipe responsavel ...","— Utilizador do Facebook, 13 maio 2022",Errado,https://observador.pt/factchecks/fact-check-bo...
1,"""Matou a enteada de cinco anos e está livre. M...","— Utilizador do Facebook, 11 maio 2020",Errado,https://observador.pt/factchecks/fact-check-fo...
2,Este rapaz chama-se Tiago Santos e é natural d...,"— Utilizador do Facebook, 25 junho 2018",Errado,https://observador.pt/factchecks/fact-check-fo...
3,"""Há 40 anos que faço pesquisas com vírus e ani...","— Utilizador do Facebook, 26 abril 2020",Errado,https://observador.pt/factchecks/fact-check-pr...
4,"""França aprova aborto até aos nove meses de ge...","— Utilizador do Facebook, 12 agosto 2020",Enganador,https://observador.pt/factchecks/fact-check-fr...


In [65]:
df.head()
df.to_csv("Observador_fact_check_dataset.csv", index=False)

In [2]:
loaded_df = pd.read_csv("Observador_fact_check_dataset.csv")
loaded_df.head()

,Text,Source,Label,URL
0,"""Policia Francesa encontra equipe responsavel ...","— Utilizador do Facebook, 13 maio 2022",Errado,https://observador.pt/factchecks/fact-check-bo...
1,"""Matou a enteada de cinco anos e está livre. M...","— Utilizador do Facebook, 11 maio 2020",Errado,https://observador.pt/factchecks/fact-check-fo...
2,Este rapaz chama-se Tiago Santos e é natural d...,"— Utilizador do Facebook, 25 junho 2018",Errado,https://observador.pt/factchecks/fact-check-fo...
3,"""Há 40 anos que faço pesquisas com vírus e ani...","— Utilizador do Facebook, 26 abril 2020",Errado,https://observador.pt/factchecks/fact-check-pr...
4,"""França aprova aborto até aos nove meses de ge...","— Utilizador do Facebook, 12 agosto 2020",Enganador,https://observador.pt/factchecks/fact-check-fr...


In [67]:
for source in set(loaded_df['Source']):
    print(source)

— Utilizador do Facebook,     13 novembro 2019
— Utilizador do Facebook,     20 abril 2022
— Site Lotus Lux,     09 julho 2019
— Inês Sousa Real,  deputada do PAN,    13 outubro 2020
— Utilizador do Facebook,     05 janeiro 2023
— Utilizador de Facebook,     23 junho 2022
— Utilizador de Facebook,    Suposta notícia de um site com frase de Mark Rutte  10 dezembro 2022
— Utilizador de Facebook,     05 dezembro 2022
— Página de Facebook "Frente Anti-Comunista Portuguesa",     23 outubro 2019
— Marcelo Rebelo de Sousa,  Presidente da República,    08 fevereiro 2017
— D. Manuel Clemente,  cardeal-patriarca de Lisba,    31 março 2022
— Utilizador de Facebook,     26 dezembro 2020
— Utilizador de Facebook,     05 setembro 2022
— Utilizador de Facebook,     06 outubro 2021
— Utilizador de Facebook,     06 novembro 2020
— André Ventura,  candidato presidencial e deputado do Chega,    07 janeiro 2021
— Utilizador do Facebook,     15 novembro 2020
— Utilizador de Facebook,     25 julho 2020
— Ut

In [38]:
loaded_df = pd.read_csv("Observador_fact_check_dataset.csv")
loaded_df.head()

,Text,Source,Label,URL
0,"""Policia Francesa encontra equipe responsavel ...","— Utilizador do Facebook, 13 maio 2022",Errado,https://observador.pt/factchecks/fact-check-bo...
1,"""Matou a enteada de cinco anos e está livre. M...","— Utilizador do Facebook, 11 maio 2020",Errado,https://observador.pt/factchecks/fact-check-fo...
2,Este rapaz chama-se Tiago Santos e é natural d...,"— Utilizador do Facebook, 25 junho 2018",Errado,https://observador.pt/factchecks/fact-check-fo...
3,"""Há 40 anos que faço pesquisas com vírus e ani...","— Utilizador do Facebook, 26 abril 2020",Errado,https://observador.pt/factchecks/fact-check-pr...
4,"""França aprova aborto até aos nove meses de ge...","— Utilizador do Facebook, 12 agosto 2020",Enganador,https://observador.pt/factchecks/fact-check-fr...


In [39]:
num_rows_with_none = len(loaded_df[loaded_df['Text'] == 'NONE'])
print(num_rows_with_none)

136


In [40]:
# Iterate over the DataFrame rows
for index, row in loaded_df.iterrows():
    if row['Text'] == "NONE":
        url = row['URL']
        print("-------------")
        print(url)
        print("-------------")
        text = "NONE"
        source = "NONE"
        label = "NONE"
        try:
            # Open webpage
            #driver.get(url)
            user_agent = UserAgent()
            headers = {'User-Agent': user_agent.random}
            page = requests.get(url, headers=headers)
            soup = bs4(page.text, 'html.parser')

            a_frase = soup.find("p", class_="factcheck-text")
            h1_elements = soup.find_all("h1")
            if a_frase:
                print("a_frase")
                text = a_frase.get_text().strip()
                source = soup.find("p", class_="factcheck-reference").get_text().strip()
                label = soup.find("div", class_="factcheck-result-text").get_text()

        except Exception as e:
            print(f"An error occurred while processing the URL: {url}")
            print(f"Error message: {e}")

        # Update the DataFrame with the extracted values
        print(text)
        print(source)
        print(label)
        loaded_df.at[index, 'Text'] = text
        loaded_df.at[index, 'Source'] = source
        loaded_df.at[index, 'Label'] = label

        time.sleep(3)

# Save the updated DataFrame to file
loaded_df.to_csv('Observador_fact_check_final_dataset.csv', index=False)

-------------
https://observador.pt/factchecks/fact-check-portugal-tem-pleno-emprego-para-os-investigadores-doutorados/
-------------
a_frase
“A minha principal preocupação neste momento é formar mais doutorados, porque a procura de doutorados é hoje bastante superior à oferta. Temos que formar mais doutores. No caso do concurso individual, não temos doutores para todas as vagas.”
— Manuel Heitor,  ministro da Ciência, Tecnologia e Ensino Superior,   em entrevista ao Público  01 fevereiro 2019
Errado
-------------
https://observador.pt/2016/02/01/decidiu-afinal-resolucao-do-banif/
-------------
NONE
NONE
NONE
-------------
https://observador.pt/especiais/debate-em-plano-inclinado-as-fugas-a-verdade-no-confronto-entre-mike-pence-e-kamala-harris/
-------------
NONE
NONE
NONE
-------------
https://observador.pt/factchecks/previsoes-das-instituicoes-internacionais-sobre-o-defice-falharam-sera-mesmo-assim-parte-ii/
-------------
a_frase
“Todas as instituições internacionais falharam nas sua

In [41]:
num_rows_with_none = len(loaded_df[loaded_df['Text'] == 'NONE'])
print(num_rows_with_none)

101


In [ ]:
# 101 Articles remain, along with 571 podcasts